<a href="https://colab.research.google.com/github/MrNerd01/models/blob/master/Assignment_3_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 3 part  2 



# Data Preprocessing :
* Download the Shakespear dataset.




In [1]:
!wget 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt'

--2019-05-31 19:03:34--  https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 2607:f8b0:400e:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘shakespeare.txt’

shakespeare.txt     100%[===================>]   1.06M  --.-KB/s    in 0.009s  

2019-05-31 19:03:39 (113 MB/s) - ‘shakespeare.txt’ saved [1115394/1115394]



In [2]:
text = open('./shakespeare.txt', 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [3]:
# Take a look at the first 250 characters in text
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



#  Data Prepocessing 
* As we can see in the above conversation there are lot of words and special characters which need to be removed such as the following :
*  Recurrent words such as "" First Citizen , All""
* special characters such as ", "," : "," ;" 
the above should be preocessed before the model is fitted.

In [0]:
import string

In [12]:
corpus = text 
bannedWords = ['First', 'Citizen:','All',',',':','.']
corpus = corpus.split()
corpus = [ word for word in corpus if word not in bannedWords]
table = str.maketrans('', '', string.punctuation)
tokens = [w.translate(table) for w in corpus]
	# remove remaining tokens that are not alphabetic
tokens = [word for word in tokens if word.isalpha()]
	# make lower case
tokens = [word.lower() for word in tokens]

print(tokens[:10])


['before', 'we', 'proceed', 'any', 'further', 'hear', 'me', 'speak', 'all', 'speak']


In [0]:
def save_doc(lines, filename):
	data = '\n'.join(lines)
	file = open(filename, 'w')
	file.write(data)
	file.close()


In [0]:
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# Create Sequences with different Combinations

In [22]:
length = 50 + 1
sequences = list()
for i in range(length, len(tokens)):
	# select sequence of tokens
	seq = tokens[i-length:i]
	# convert into a line
	line = ' '.join(seq)
	# store
	sequences.append(line)
print('Sequences: %d' % len(sequences))
#Save sequences for future use.
save_doc(sequences,'sequences.txt')

Sequences: 202166


In [0]:
from numpy import array
from pickle import dump
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding,Flatten,Bidirectional

In [25]:
in_filename = 'sequences.txt'
doc = load_doc(in_filename)
lines = doc.split('\n')[:10000]
print(lines[:100])

['before we proceed any further hear me speak all speak speak you are all resolved rather to die than to famish all resolved resolved first you know caius marcius is chief enemy to the people all we knowt we knowt let us kill him and well have corn at our own', 'we proceed any further hear me speak all speak speak you are all resolved rather to die than to famish all resolved resolved first you know caius marcius is chief enemy to the people all we knowt we knowt let us kill him and well have corn at our own price', 'proceed any further hear me speak all speak speak you are all resolved rather to die than to famish all resolved resolved first you know caius marcius is chief enemy to the people all we knowt we knowt let us kill him and well have corn at our own price ist', 'any further hear me speak all speak speak you are all resolved rather to die than to famish all resolved resolved first you know caius marcius is chief enemy to the people all we knowt we knowt let us kill him and we

In [0]:

tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)

In [27]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

2236


In [0]:
sequences = array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

# RNN using uni - directional LSTM.

In [29]:
# define model
model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 50, 50)            111800    
_________________________________________________________________
lstm_5 (LSTM)                (None, 50, 100)           60400     
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_4 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_5 (Dense)              (None, 2236)              225836    
Total params: 488,536
Trainable params: 488,536
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
10000/10000 [==============================] - 13s 1ms/step - loss: 6.8157 - acc: 0.0354
Epoch 2/10
10000/10000 [=======

In [0]:
# save the model to file
model.save('model.h5')
# save the tokenizer
dump(tokenizer, open('tokenizer.pkl', 'wb'))

# Prediction  for uni- directional LSTM.

In [0]:
from pickle import load
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences

doc = load_doc('sequences.txt')
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
model = load_model('model.h5')
tokenizer = load(open('tokenizer.pkl', 'rb'))
seed_text = 'before'

In [33]:
result = list()
in_text = seed_text
	# generate a fixed number of words
for _ in range(10):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
print(' '.join(result))

the people and the people and the people and the


# RNN using BiDirectional LSTM.

In [41]:
from keras.layers import Activation, Dense,Dropout

model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=seq_length))
model.add(Bidirectional(LSTM(64)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='sigmoid'))
model.summary()

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 50, 128)           286208    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 128)               98816     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               12900     
_________________________________________________________________
dense_9 (Dense)              (None, 2236)              225836    
Total params: 623,760
Trainable params: 623,760
Non-trainable params: 0
_________________________________________________________________


In [0]:
# save the model to file
model.save('modelbi.h5')

In [43]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, batch_size=128, epochs=10)

Epoch 1/10
10000/10000 [==============================] - 19s 2ms/step - loss: 7.0750 - acc: 0.0380
Epoch 2/10
10000/10000 [==============================] - 14s 1ms/step - loss: 6.4869 - acc: 0.0415
Epoch 3/10
10000/10000 [==============================] - 14s 1ms/step - loss: 6.3240 - acc: 0.0418
Epoch 4/10
10000/10000 [==============================] - 14s 1ms/step - loss: 6.2102 - acc: 0.0418
Epoch 5/10
10000/10000 [==============================] - 14s 1ms/step - loss: 6.1372 - acc: 0.0418
Epoch 6/10
10000/10000 [==============================] - 14s 1ms/step - loss: 6.0794 - acc: 0.0418
Epoch 7/10
10000/10000 [==============================] - 14s 1ms/step - loss: 6.0092 - acc: 0.0418
Epoch 8/10
10000/10000 [==============================] - 14s 1ms/step - loss: 5.9434 - acc: 0.0418
Epoch 9/10
10000/10000 [==============================] - 14s 1ms/step - loss: 5.8803 - acc: 0.0418
Epoch 10/10
10000/10000 [==============================] - 14s 1ms/step - loss: 5.8096 - acc: 0.0419

In [44]:
doc = load_doc('sequences.txt')
lines = doc.split('\n')
seq_length = len(lines[0].split()) - 1
model = load_model('modelbi.h5')
tokenizer = load(open('tokenizer.pkl', 'rb'))
seed_text = 'before'

/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [45]:
result = list()
in_text = seed_text
	# generate a fixed number of words
for _ in range(10):
		# encode the text as integer
		encoded = tokenizer.texts_to_sequences([in_text])[0]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict probabilities for each word
		yhat = model.predict_classes(encoded, verbose=0)
		# map predicted word index to word
		out_word = ''
		for word, index in tokenizer.word_index.items():
			if index == yhat:
				out_word = word
				break
		# append to input
		in_text += ' ' + out_word
		result.append(out_word)
print(' '.join(result))

garners oaken oaken spoons spoons percussion spoons percussion countrymen renowned


# Unidirectional LSTM only preserves information of the past because the only inputs it has seen are from the past.
**OUTPUT of uni-directional LSTM** :-  "the people and the people and the people and the"

# Using bidirectional will run your inputs in two ways, one from past to future and one from future to past 
**OUTPUT of BiDirectional LSTM** :-"" garners oaken oaken spoons spoons percussion spoons percussion countrymen renowned"